In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow


from tensorflow.keras.models import Sequential
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation


from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from tensorflow.keras.preprocessing import image
from keras.optimizers import Adam


from tensorflow import keras
from tensorflow.keras import layers

from sklearn.cluster import KMeans
from scipy import spatial
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [ ]:

class Dataset(object):
    def __init__(self, data_path):
        self.data_path = data_path
        assert os.path.exists(self.data_path), 'Insert a valid path!'

        # get class list
        self.data_classes = os.listdir(self.data_path)

        # init mapping dict
        self.data_mapping = {}

        # populate mapping dict
        for c, c_name in enumerate(self.data_classes):
            temp_path = os.path.join(self.data_path, c_name)
            temp_images = os.listdir(temp_path)

            for i in temp_images:
                img_tmp = os.path.join(temp_path, i)

                if img_tmp.endswith('.jpg'):
                    if c_name == 'distractor':
                        self.data_mapping[img_tmp] = -1
                    else:
                        self.data_mapping[img_tmp] = int(c_name)

        print('Loaded {:d} from {:s} images'.format(len(self.data_mapping.keys()),
                                                    self.data_path))

    def get_data_paths(self):
        # returns a list of imgpaths and related classes
        images = []
        classes = []
        for img_path in self.data_mapping.keys():
            if img_path.endswith('.jpg'):
                images.append(img_path)
                classes.append(self.data_mapping[img_path])
        return images, np.array(classes)


    def num_classes(self):
        # returns number of classes of the dataset
        return len(self.data_classes)


In [ ]:
# we get data_path
data_path = '/content/drive/MyDrive/AML_Challange/dataset'
# we define training_path
training_path = os.path.join(data_path, 'training')

# we define validation path, query and gallery
validation_path = os.path.join(data_path, 'validation')
gallery_path = os.path.join(validation_path, 'gallery')
query_path = os.path.join(validation_path, 'query')


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#All images are rescaled by 1./255
train_datagen = ImageDataGenerator( rescale= 1.0/255.)
validation_datagen = ImageDataGenerator( rescale= 1.0/255.)
query_datagen = ImageDataGenerator (rescale= 1.0/255.)
gallery_datagen = ImageDataGenerator( rescale= 1.0/255.)



train_generator =train_datagen.flow_from_directory(training_path,                                                
                                                    batch_size=20,
                                                   
                                                    target_size = (224,224))

                                                   

validation_generator = validation_datagen.flow_from_directory(validation_path,
                                                       batch_size=20,
                                                      class_mode =None,
                                                       target_size = (224,224))
                                                        

query_generator = query_datagen.flow_from_directory(query_path, 
                                                        batch_size=20,
                                                        class_mode =None,
                                                        target_size = (224,224))
                                                       

gallery_generator = gallery_datagen.flow_from_directory(gallery_path,
                                                        batch_size=20,
                                                        class_mode =None,
                                                        target_size = (224,224))
                                               

Found 49553 images belonging to 22 classes.
Found 604 images belonging to 2 classes.
Found 70 images belonging to 14 classes.
Found 534 images belonging to 15 classes.


In [ ]:
 
model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape =(224, 224, 3), padding='same'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(1024, activation='relu'),
      tf.keras.layers.Dense(22)])



NameError: ignored

In [ ]:
model.compile(optimizer='adam',
              #loss='sparse_categorical_crossentropy',
              loss = keras.losses.categorical_crossentropy,
              metrics=['accuracy'])
             

In [ ]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 224, 224, 16)      448       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 32)      4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 56, 56, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 28, 28, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 50176)            

In [ ]:
history = model.fit(
        train_generator,
        steps_per_epoch=50,
        epochs=15,
        validation_data=validation_generator,
        verbose=1)

Epoch 1/15
50/50 [==============================] - 591s 12s/step - loss: 10.9764 - accuracy: 0.1860 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/15
50/50 [==============================] - 562s 11s/step - loss: 11.0892 - accuracy: 0.2110 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/15
50/50 [==============================] - 590s 12s/step - loss: 11.0409 - accuracy: 0.1850 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/15
50/50 [==============================] - 580s 12s/step - loss: 10.7669 - accuracy: 0.1740 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/15
50/50 [==============================] - 580s 12s/step - loss: 10.9120 - accuracy: 0.1950 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/15
50/50 [==============================] - 550s 11s/step - loss: 10.6702 - accuracy: 0.1690 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/15
50/50 [==============================] - 554s 11s/step - loss: 10.8958 - accuracy: 

In [ ]:
scores = model.evaluate(train_generator, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

NameError: ignored

CHALLENGE



In [ ]:
test_path= os.path.join(data_path, "test")

test_gallery_path = os.path.join(test_path, 'gallery')
test_query_path = os.path.join(test_path, 'query')

def imag_load(path) : 
  temp_images = os.listdir(path)
  lista = []
  for i in temp_images:
    img_tmp = os.path.join(path, i)
    lista.append(img_tmp)
  return lista

gallery = imag_load(gallery_path)
query = imag_load(query_path)

query_dataset = convert_to_array(query)
gallery_dataset = convert_to_array(gallery)